In [13]:
# Libraries

import pandas as pd
import random

In [14]:
# Retrieves image, symbol and paytable information from Excel file.

Symbols = []
PayTable = []
ReelSets = []

# Excel File Path
file_name = 'SlotElementGuide_Parsheet.ods'

# Reading data from excel file
data_symbols = pd.read_excel(file_name, sheet_name = 'Symbols')
data_paytable = pd.read_excel(file_name, sheet_name = 'Pays')
data_reelset = pd.read_excel(file_name, sheet_name = 'Reelsets')

Symbols = data_symbols.iloc[2:14, 0].tolist()

PayTable5 = data_paytable.iloc[:, 5].dropna().tolist()
PayTable4 = data_paytable.iloc[:, 4].dropna().tolist()
PayTable3 = data_paytable.iloc[:, 3].dropna().tolist()
PayTable2 = data_paytable.iloc[:, 2].dropna().tolist()
PayTable1 = data_paytable.iloc[:, 1].dropna().tolist()
PayTable0 = data_paytable.iloc[:, 0].dropna().tolist()

# ScatPayTable = data_paytable.iloc[8, :5].tolist()

Reel1 = data_reelset.iloc[:, 1].dropna().tolist()
Reel2 = data_reelset.iloc[:, 2].dropna().tolist()
Reel3 = data_reelset.iloc[:, 3].dropna().tolist()
Reel4 = data_reelset.iloc[:, 4].dropna().tolist()
Reel5 = data_reelset.iloc[:, 5].dropna().tolist()

PayTable = [PayTable5, PayTable4, PayTable3, PayTable2, PayTable1, PayTable0]
ReelSet = [Reel1, Reel2, Reel3, Reel4, Reel5]
Free_Spin_Counts = {
    3: 10,  # Number of spins for 3 scatters
    4: 15,  # Number of spins for 4 scatters
    5: 25   # Number of spins for 5 scatters
}


In [15]:
class Spin:
    def __init__(self, ReelSet, bet):
        self.ReelSet = ReelSet
        self.bet = bet
        self.spin_result = []
        self.total_bet = 0  

    def execute(self, bet):
        """ Choose a random index for each reel and create a sequential spin. """
        self.spin_result = []
        for reel in self.ReelSet:
            # Choose a random index
            random_index = random.randint(0, len(reel) - 1)
            
            # Edit index limits with mod
            left_index = (random_index - 1) % len(reel)
            middle_index = random_index
            right_index = (random_index + 1) % len(reel)
            
            # Generate sequential spin results
            result_column = [
                reel[left_index],
                reel[middle_index],
                reel[right_index]
            ]
            self.spin_result.append(result_column)

        self.total_bet += bet

        return self.spin_result
    

In [16]:
# Create Spin and Win objects
spin_machine = Spin(ReelSet, bet = 1)

# Get spin count from user
spin_count = int(input("How many spins do you want to create? "))

In [17]:
class Win:
    def __init__(self, paytable, symbols, wild_symbol="Wild", scatter_symbol = "Scat"):
        
        self.paytable = paytable
        self.symbols = symbols
        self.wild_symbol = wild_symbol
        self.scatter_symbol = scatter_symbol
        self.total_win = 0  

        self.basegame_line_win = 0
        self.basegame_scat_win = 0
        self.freespin_line_win = 0
        self.freespin_scat_win = 0

        self.total_basegame_line_win = 0  
        self.total_basegame_scatter_win = 0
        self.total_freespin_line_win = 0
        self.total_freespin_scat_win = 0
        self.free_spin_count = 0  

        self.payout_distribution = {symbol: [0, 0, 0, 0, 0] for symbol in symbols}  # 1-5 wins
        self.total_spins = 0

        self.total_scatter_triggers = 0
        self.total_freespins_given = 0

    def line_win_check(self, spin_result, mode = "basegame"):
        self.total_spins += 1

        middle_symbols = [reel[1] for reel in spin_result]  # The middle symbol of each wheel.
        line_win_symbol = middle_symbols[0]  # Base on first symbol
        same_symbol_count = 1

        if line_win_symbol == "Scat":
            return 0

        for symbol in middle_symbols[1:]:
            if symbol == line_win_symbol or symbol == self.wild_symbol or line_win_symbol == self.wild_symbol:
                same_symbol_count += 1
                if line_win_symbol == self.wild_symbol and symbol != self.wild_symbol:
                    line_win_symbol = symbol
            else:
                break

        symbol_index = self.symbols.index(line_win_symbol) if line_win_symbol in self.symbols else -1
        basegame_line_win = 0
        
        if symbol_index != -1 and same_symbol_count < len(self.paytable):
            basegame_line_win = self.paytable[same_symbol_count][symbol_index]
            
            # Add to total and distribution only in "basegame" mode
            if mode == "basegame":
                self.total_basegame_line_win += basegame_line_win
                self.payout_distribution[line_win_symbol][same_symbol_count - 1] += basegame_line_win

            # If in "freespin" mode, add only freespin winnings
            elif mode == "freespin":
                self.total_freespin_line_win += basegame_line_win

        return basegame_line_win
    
    def scatter_win_check(self, spin_result, mode = "basegame"):   # Base Game Scatter Returns Win.

        all_symbols = [symbol for reel in spin_result for symbol in reel]
        scatter_count = all_symbols.count(self.scatter_symbol)
        scatter_index = self.symbols.index(self.scatter_symbol) if self.scatter_symbol in self.symbols else -1

        basegame_scat_win = 0
        if scatter_index != -1 and scatter_count < len(self.paytable):
            basegame_scat_win = self.paytable[scatter_count][scatter_index]

            # Add to total and distribution only in "basegame" mode
            if mode == "basegame":
                self.total_basegame_scatter_win += basegame_scat_win

            # If in "freespin" mode, add only freespin winnings
            elif mode == "freespin":
                self.total_freespin_scat_win += basegame_scat_win
        

        return basegame_scat_win
    
    def freespin_win_check(self, spin_result, freespin_round=1):
        all_symbols = [symbol for reel in spin_result for symbol in reel]
        scatter_count = all_symbols.count(self.scatter_symbol)

        if scatter_count in Free_Spin_Counts:
            # Increase scatter trigger count
            self.total_scatter_triggers += 1

            total_freespin_win = 0
            total_fs_scat_win = 0

            # Increase the number of triggered free spins
            free_spins = Free_Spin_Counts[scatter_count]
            self.total_freespins_given += free_spins  # Increase the total number of free spins awarded

            freespin_line_win = 0
            total_fs_line_win = 0
            current_spin = 0  # Current freespin number

            while free_spins > 0:  # Continue until the free spins count reaches zero.
                current_spin += 1
                spin_result = spin_machine.execute(bet=0)

                freespin_line_win = self.line_win_check(spin_result, mode="freespin")
                total_fs_line_win += freespin_line_win

                # New scatter control
                freespin_scat_win = 0
                new_scatter_count = [symbol for reel in spin_result for symbol in reel].count(self.scatter_symbol)

                if new_scatter_count in Free_Spin_Counts:
                    additional_spins = Free_Spin_Counts[new_scatter_count]
                    free_spins += additional_spins

                    # Add new freespin trigger to counters
                    self.total_scatter_triggers += 1  # New trigger
                    self.total_freespins_given += additional_spins  # Number of free spins added

                    freespin_scat_win = self.scatter_win_check(spin_result, mode="freespin")
                    total_fs_scat_win += freespin_scat_win

                # One spin completed
                free_spins -= 1

            total_freespin_win = total_fs_line_win + total_fs_scat_win

            return total_freespin_win


In [18]:
win_checker = Win(PayTable, Symbols)
spin_win_distribution = {}

# Run spins and calculate Line RTP
for spin_index in range(spin_count):
    spin_result = spin_machine.execute(bet = 1)
    line_win = win_checker.line_win_check(spin_result)
    scatter_win = win_checker.scatter_win_check(spin_result)
    freespin_win = win_checker.freespin_win_check(spin_result)
    # print("Spin Result: ", spin_result)
    # print("Line Win: ", line_win)
    # print("Scatter Win: ", scatter_win)

    line_win = line_win if line_win is not None else 0
    scatter_win = scatter_win if scatter_win is not None else 0
    freespin_win = freespin_win if freespin_win is not None else 0
    spin_win = line_win + scatter_win + freespin_win

    # Calculate total earnings
    total_spin_win = line_win + scatter_win + freespin_win

    # Add the winning amount to the distribution dictionary
    if total_spin_win not in spin_win_distribution:
        spin_win_distribution[total_spin_win] = 0
    spin_win_distribution[total_spin_win] += 1  # This gain occurred once again
    
sorted_spin_win_distribution = dict(sorted(spin_win_distribution.items(), key=lambda x: x[1], reverse=True))
total_win = (win_checker.total_basegame_line_win + win_checker.total_basegame_scatter_win + win_checker.total_freespin_line_win + win_checker.total_freespin_scat_win)

# Total RTP result
print("Total Bet:", spin_machine.total_bet, "\n")
print("Base Game Line Win:", win_checker.total_basegame_line_win )
print(f"Base Game Line RTP: {(win_checker.total_basegame_line_win)/spin_machine.total_bet*100:.2f}%", "\n")

print("Base Game Scatter Win:", win_checker.total_basegame_scatter_win)
print(f"Base Game Scatter RTP: {win_checker.total_basegame_scatter_win/spin_machine.total_bet*100:.2f}%", "\n")

print("FreeSpin Line Win:", win_checker.total_freespin_line_win)
print(f"FreeSpin Line RTP:{win_checker.total_freespin_line_win/spin_machine.total_bet*100:.2f}%", "\n")

print("FreeSpin Scatter Win:", win_checker.total_freespin_scat_win)
print(f"FreeSpin Scatter RTP:{win_checker.total_freespin_scat_win/spin_machine.total_bet*100:.2f}%", "\n")

print(f"Total Scatter Trigger Count: {win_checker.total_scatter_triggers}")
print(f"Total Number of Free Spins Given: {win_checker.total_freespins_given}", "\n")

# Average Number of Free Spins
average_freespin_count = win_checker.total_freespins_given / win_checker.total_scatter_triggers
print(f"Average Number of Free Spins: {average_freespin_count}")
print(f"Expected Number of Free Spins: {win_checker.total_freespins_given/spin_machine.total_bet}", "\n")

print("Total Win:", total_win)
print(f"Final RTP: {total_win/spin_machine.total_bet*100:.2f}%")


Total Bet: 1000000 

Base Game Line Win: 747170
Base Game Line RTP: 74.72% 

Base Game Scatter Win: 95490
Base Game Scatter RTP: 9.55% 

FreeSpin Line Win: 79455
FreeSpin Line RTP:7.95% 

FreeSpin Scatter Win: 9790
FreeSpin Scatter RTP:0.98% 

Total Scatter Trigger Count: 9936
Total Number of Free Spins Given: 102080 

Average Number of Free Spins: 10.273752012882447
Expected Number of Free Spins: 0.10208 

Total Win: 931905
Final RTP: 93.19%


In [19]:
import ezodf

def save_spin_distribution_to_ods(spin_win_distribution, filename="spin_win_distribution.ods"):

    # Create a new .ods document
    spreadsheet = ezodf.newdoc(doctype="ods", filename=filename)

    # Add a new sheet
    sheet = ezodf.Sheet("Spin Win Distribution", size=(len(spin_win_distribution) + 1, 2))
    spreadsheet.sheets += sheet

    # Add headers
    sheet[0, 0].set_value("Win Amount")
    sheet[0, 1].set_value("Spin Count")

    # Populate the sheet with data
    for row_index, (win_amount, spin_count) in enumerate(spin_win_distribution.items(), start=1):
        sheet[row_index, 0].set_value(win_amount)
        sheet[row_index, 1].set_value(spin_count)

    # Save the .ods file
    spreadsheet.save()
    print(f"Data successfully saved to {filename}!")

# save_spin_distribution_to_ods(spin_win_distribution)